In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
def write_answer(assignment_N, answer):
    with open("rank_{}.txt".format(assignment_N), "w") as fout:
        fout.write("{:.2f} {:.2f} {:.2f} {:.2f}".format(answer[0], answer[1], answer[2], answer[3]))

In [3]:
def read_file(filename):
    viewed_items = Counter()
    buyed_items = Counter()
    
    with open(filename, 'rt') as f:
        user_count = 0
        for user, line in enumerate(f.readlines()):
            viewed, buyed = line.strip().split(';')
            
            #if len(buyed)==0:
            #    continue
            
            user_count += 1
                
            viewed = viewed.split(',')
            buyed = buyed.split(',')
            
            for item in viewed:
                viewed_items[item] += 1
            for item in buyed:
                buyed_items[item] += 1
            
            #for item in viewed:
            #    if item in viewed_items:
            #        viewed_items[item] = (viewed_items[item][0] + 1, viewed_items[item][1].add(user))
            #    else:
            #        viewed_items[item] = (1, set(user))
            
    return viewed_items, buyed_items, user_count

In [4]:
def read_file_test(filename):
    users = []
    
    with open(filename, 'rt') as f:
        for line in f.readlines():
            viewed, buyed = line.strip().split(';')
            
            if len(buyed) ==0:
                continue
            
            viewed = viewed.split(',')
            buyed = buyed.split(',')
            
            
            users.append((viewed,buyed))
    return users

In [30]:
viewed_count, buyed_count, user_count = read_file('coursera_sessions_train.txt')
train_data = read_file_test('coursera_sessions_train.txt')
test_data = read_file_test('coursera_sessions_test.txt')

In [89]:
def predict(viewed, k, method='popular'):
    unique_viewed = np.array(viewed)[np.sort(np.unique(viewed, return_index=True)[1])]
    C = Counter(viewed)
    k = min(len(viewed), k)    
    ratings = []
    
    for item in unique_viewed:
        if method == 'popular':
            #ratings.append(-viewed_count[item]*C[item])
            ratings.append(-viewed_count[item])
        else:
            #ratings.append(-buyed_count[item]*C[item])
            ratings.append(-buyed_count[item])
    
    sorted_items = np.argsort(ratings, kind='mergesort')
    return list(unique_viewed[sorted_items])[0:k]

In [90]:
def get_user_precision(x):
    return precision(predict(test_data[x][0], 5), test_data[x][1], 5)

In [91]:
def precision(predicted, buyed, k):
    rb = [x for x in buyed if x in predicted]
    return len(rb)/float(k)

In [92]:
def recall(predicted, buyed):
    rb = [x for x in buyed if x in predicted]
    return len(rb)/float(len(buyed))

In [93]:
def get_stats(data,method='popular'):
    ar1 = np.mean([recall(predict(user[0], 1, method=method), user[1]) for user in data])
    ap1 = np.mean([precision(predict(user[0], 1, method=method), user[1], 1) for user in data])
    ar5 = np.mean([recall(predict(user[0], 5, method=method), user[1]) for user in data])
    ap5 = np.mean([precision(predict(user[0], 5, method=method), user[1], 5) for user in data])
    
    return ar1, ap1, ar5, ap5

In [94]:
train_popular = get_stats(train_data, 'popular')
test_popular = get_stats(test_data, 'popular')
train_purch = get_stats(train_data, 'purch')
test_purch = get_stats(test_data, 'purch')

In [95]:
write_answer(1, train_popular)
write_answer(2, test_popular)
write_answer(3, train_purch)
write_answer(4, test_purch)

In [88]:
def mprint(user, method='popular'):
    print('Viewed: {}, Buyed: {}'.format(user[0], user[1]))
    print("Recommended {}, precision@1: {:.2f}".format())
    print('Recommended {}, precision@1: {:.2f}'.format(predict(user[0], 1, method=method),
                                               precision(predict(user[0], 1, method=method), user[1], 1)))
    print('Recommended {}, precision@1: {:.2f}'.format(predict(user[0], 1, method=method),
                                               recall(predict(user[0], 1, method=method), user[1]))
    print('Recommended {}, precision@1: {:.2f}'.format(predict(user[0], 5, method=method),
                                               precision(predict(user[0], 5, method=method), user[1], 5)))
    print('Recommended {}, precision@1: {:.2f}'.format(predict(user[0], 1, method=method),
                                               precision(predict(user[0], 1, method=method), user[1], 1)))
        
        precision(predict(user[0], 5, method=method), user[1], 5),
        recall(predict(user[0], 5, method=method), user[1])))

SyntaxError: invalid syntax (<ipython-input-88-8c9534a0e141>, line 8)

In [ ]:
for user in train_data[0:10]:
    mprint(user)